In [339]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [340]:
import mlflow
import pandas as pd

from cookiemonster.utils import LOGS_PATH

In [341]:
from mlflow.tracking import MlflowClient

tracking_uri = str(LOGS_PATH.joinpath("mlflow"))

mlflow.set_tracking_uri(tracking_uri)
client = MlflowClient(
    tracking_uri=tracking_uri
)

In [342]:

# experiment_name="bias_detection_09-09_11-39"
# experiment_name="bias_detection_09-09_18-06" # Shuffled?
# experiment_name="bias_detection_09-10_11-01"
# experiment_name="bias_detection_09-10_11-14" # 60. Good but CM has small bias.
# experiment_name="bias_detection_09-10_11-27" # 45. Good
# experiment_name="bias_detection_09-10_11-38" # 120 Barely more than IPA (more overshooting?) and starts to go in steady state, crossing ARA. But CM itself is not much better than IPA.
# experiment_name="bias_detection_09-10_16-17" # 120 with more schedules, 200. Just higher error in steady state. Better strategy would be to wait.

# experiment_name="bias_detection_09-11_15-55" # 60, fixed sensitivity.
# experiment_name="bias_detection_09-11_18-17" # 60, 400 queries
# experiment_name="bias_detection_09-11_20-46" # 60, 250 queries

# experiment_name="bias_detection_09-15_13-57" # 60, 400 queries, 0.01 impressions per day
# experiment_name="bias_detection_09-15_20-07" # 120, 800 queries. Too long tail.

# experiment_name="bias_detection_09-12_22-18" # 60, 400 queries. 0.1 & 0.1. Overleaf.


# experiment_name="bias_detection_criteo_09-10_13-46" # Old Criteo
# experiment_name="bias_detection_criteo_09-10_15-32" # Actually PACG. CM stays below 0.05 the whole time. But not too bad? Not much overestimation. Slow exp though.

# experiment_name="bias_detection_patcg_09-15_13-03" # PATCG, new
# experiment_name="bias_detection_criteo_09-15_19-25" # Criteo, new

experiment_name="bias_detection_09-19_10-06"

# experiment_name="bias_detection_patcg_09-19_11-11"

runs = mlflow.search_runs(experiment_names=[experiment_name])


# experiments_names = [f"{experiment_name} ({i})" for i in range(7)]
# runs = mlflow.search_runs(experiment_names=experiments_names, search_all_experiments=True)


run_ids = list(runs.run_id)
run_ids

# run_ids = [
#     "1cda7a58d57b459e8f751b8191877c31",
#     "d53af937a7d14829bff4f44edf86a773",
#     "acc488ccc09c41638eb8b2a6c1f6dc9c",
#     "7a0bb2058c3441eda6a41227527d98e3",
#     "33312ed38ddc4d6ba0812cfbeddf397e",
#     "938f81a4c4f44784b86c338d1eb6a27a",
# ]

['420e01aa58554fceb1c95cfa7f636727',
 '4fce2a35e734444c979ec3ff04d2806b',
 '58199c21504741b380af3e5efe6aa657',
 '4e27c18d2e124c83a1eb76651b0de93c',
 'e1c94336ba204d1cb4e461efbfe7ec72',
 'df4957031eb3480aa6d927ca7e07935b',
 'c72a9131b1b8474388ccab7747eaf389',
 '28da23ca6bc84f45be194b16592cd1f7',
 '6fb76134935743ba957171df673f3427']

In [343]:
data = []
metric_names = ["rmsre", "rmsre_prediction"]
param_names = ["user.baseline", "user.bias_detection_knob", "dataset.num_days_per_epoch", "user.is_monotonic_scalar_query"]
# param_names = ["user.baseline", "user.bias_detection_knob", "dataset.num_days_per_epoch"]


for run_id in run_ids:
    
    run_params = client.get_run(run_id).data.params
    
    metric_histories = {}
    for metric_name in metric_names:
        metric_histories[metric_name] = client.get_metric_history(run_id, metric_name)
    
    for step in range(len(metric_histories[metric_names[0]])):
        step_data = {
            "step": step,
        }
        for metric_name in metric_names:
            try:
                step_data[metric_name] = metric_histories[metric_name][step].value
            except IndexError:
                step_data[metric_name] = None
                    
    
        for param_name in param_names:
            step_data[param_name] = run_params[param_name]
    
        data.append(step_data)

df = pd.DataFrame(data)
df

,step,rmsre,rmsre_prediction,user.baseline,user.bias_detection_knob,dataset.num_days_per_epoch,user.is_monotonic_scalar_query
0,0,0.015612,NaN,cookiemonster,0,7,True
1,1,0.015596,NaN,cookiemonster,0,7,True
2,2,0.015604,NaN,cookiemonster,0,7,True
3,3,0.015565,NaN,cookiemonster,0,7,True
4,4,0.015510,NaN,cookiemonster,0,7,True
...,...,...,...,...,...,...,...
3595,395,1.000122,65.587595,cookiemonster_base,0.5,7,True
3596,396,1.000122,86.423898,cookiemonster_base,0.5,7,True
3597,397,1.000121,79.145506,cookiemonster_base,0.5,7,True
3598,398,1.000122,793.386361,cookiemonster_base,0.5,7,True


In [344]:
df["is_monotonic_scalar_query"] = df["user.is_monotonic_scalar_query"]
df["bias_detection_knob"] = df["user.bias_detection_knob"]
df["num_days_per_epoch"] = df["dataset.num_days_per_epoch"]

def get_baseline_name(row):
    if row["user.bias_detection_knob"] == "0":
        return row["user.baseline"]
    return row["user.baseline"] + "_bias"

df["baseline"] = df.apply(get_baseline_name, axis=1)



In [345]:
is_monotonic_scalar_query="True"

# bias_detection_knob="2" # Better for 0.01 imp
# rmsre_prediction_cutoff = 0.2

# PATCG
bias_detection_knob="3"
rmsre_prediction_cutoff = 0.1
y_range = [-3, -0.5]

# micro
bias_detection_knob="0.5"
rmsre_prediction_cutoff = 0.05
y_range = [-2, 0.5]

# Criteo?
# bias_detection_knob="2"
# rmsre_prediction_cutoff = 0.05
# y_range = [-2, 0.5]

query_df = df.query(f"(bias_detection_knob == '0' or bias_detection_knob == '{bias_detection_knob}') and num_days_per_epoch == '7' and is_monotonic_scalar_query == '{is_monotonic_scalar_query}'")
# query_df = df.query(f"(bias_detection_knob == '0' or bias_detection_knob == '{bias_detection_knob}') and num_days_per_epoch == '7'")


In [346]:
query_df.baseline.unique()

array(['cookiemonster', 'ipa', 'cookiemonster_base', 'cookiemonster_bias',
       'cookiemonster_base_bias'], dtype=object)

In [347]:
query_df.bias_detection_knob.unique()

array(['0', '0.5'], dtype=object)

In [348]:
num_queries = query_df.step.max() + 1
num_queries

400

In [349]:
query_df.query("baseline == 'ipa'")

,step,rmsre,rmsre_prediction,user.baseline,user.bias_detection_knob,dataset.num_days_per_epoch,user.is_monotonic_scalar_query,is_monotonic_scalar_query,bias_detection_knob,num_days_per_epoch,baseline
400,0,0.015612,NaN,ipa,0,7,True,True,0,7,ipa
401,1,0.015596,NaN,ipa,0,7,True,True,0,7,ipa
402,2,0.015604,NaN,ipa,0,7,True,True,0,7,ipa
403,3,0.015565,NaN,ipa,0,7,True,True,0,7,ipa
404,4,0.015510,NaN,ipa,0,7,True,True,0,7,ipa
...,...,...,...,...,...,...,...,...,...,...,...
795,395,NaN,NaN,ipa,0,7,True,True,0,7,ipa
796,396,NaN,NaN,ipa,0,7,True,True,0,7,ipa
797,397,NaN,NaN,ipa,0,7,True,True,0,7,ipa
798,398,NaN,NaN,ipa,0,7,True,True,0,7,ipa


In [350]:
from plotting.macros import *
from plotting.plot_template import *


In [351]:
cdf_args = {
        "df": query_df,
        "metric": "rmsre",
        "x_axis": None,
        "x_axis_title": RMSRE_CDF_X,
        "y_axis_title": RMSRE_Y_LOG,
        "ordering": None,
        "log_y": True,
        "x_range": [1, 100],
        "showlegend": True,
        "marker_pos": 0.98,
        "baselines_order": BIAS_BASELINES_ORDER,
        "y_range": y_range,
        
    }

In [352]:
def truncate_rmsre_prediction(row):
    if row["rmsre_prediction"] is None or row["rmsre_prediction"] < 1:
        return row["rmsre_prediction"]
    return 1

query_df["rmsre_prediction"] = query_df.apply(truncate_rmsre_prediction, axis=1)

/tmp/ipykernel_300942/2386874952.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [353]:

def get_unsorted_trace(query_df, baseline, metric, unsorted_metric, color):
    csv_name = csv_mapping[baseline]
    group = query_df.query("baseline == @csv_name")
    group = group.sort_values(by=[metric])
    len_values = group.shape[0]
    start = 1

    group.dropna(inplace=True, subset=[metric])
    stop = group.shape[0]
    values = group[unsorted_metric].values
    cumulative_probabilities = np.arange(start, stop + 1) / float(len_values) * 100

    trace_unsorted = go.Scatter(
            x=cumulative_probabilities,
            y=values,
            legendgroup=baseline,
            name=f"{baseline} (estimation of error)",
            # marker_color=color_discrete_map[baseline],
            marker_color=color,
            marker_symbol=symbol_map[baseline],
            showlegend=True,
            mode="lines",
            line=dict(dash=lines_map[baseline]),
        )
    return trace_unsorted

baseline = COOKIEMONSTER_BIAS
metric = "rmsre"
unsorted_metric = "rmsre_prediction"
cm_color = "rgba(128, 0, 128, 0.2)"
trace_unsorted_cm = get_unsorted_trace(query_df, baseline, metric, unsorted_metric, cm_color)




baseline = COOKIEMONSTER_BASE_BIAS
ara_color = "rgba(0, 128, 128, 0.2)"
trace_unsorted_ara = get_unsorted_trace(query_df, baseline, metric, unsorted_metric, ara_color)


# trace_cutoff = go.Scatter(
#         x=cumulative_probabilities,
#         y=[rmsre_prediction_cutoff]*len(cumulative_probabilities),
#         legendgroup=baseline,
#         name=f"{baseline} (RMSRE bound cutoff)",
#         # marker_color=color_discrete_map[baseline],
#         marker_color="black",
#         marker_symbol=symbol_map[baseline],
#         showlegend=True,
#         mode="lines",
#         line=dict(dash=lines_map[baseline], width=1),
#     )

In [354]:
full_cdfs = go.Figure()
full_cdfs.add_trace(trace_unsorted_cm)
full_cdfs.add_trace(trace_unsorted_ara)

# full_cdfs.add_trace(trace_cutoff)
for trace in cdf(**cdf_args):
    full_cdfs.add_trace(trace)
full_cdfs.update_layout(title=f"CDF of RMRSRE. Bias detection knob: {bias_detection_knob}. #queries {num_queries}")
full_cdfs

In [355]:

def apply_bias_detection(row):
    

    if rmsre_prediction_cutoff == 0.5:
        print("Got {}".format(row["rmsre_prediction"]))
    
    if row["rmsre_prediction"] and row["rmsre_prediction"] > rmsre_prediction_cutoff:
        if rmsre_prediction_cutoff == 0.5:
            print("dropping it")

        return None
    return row["rmsre_original"]

chopped_df = query_df.copy()
chopped_df["rmsre_original"] = chopped_df["rmsre"]
chopped_df["rmsre"] = chopped_df.apply(apply_bias_detection, axis=1)
chopped_df.drop(columns=["rmsre_prediction"], inplace=True)

In [356]:
cut_cdf_args = cdf_args.copy()
cut_cdf_args["df"] = chopped_df
cut_cdf_args["showlegend"] = False

cut_cdfs_fig = go.Figure()
for trace in cdf(**cut_cdf_args):
    cut_cdfs_fig.add_trace(trace)
cut_cdfs_fig.update_layout(title=f"CDF of RMSRE. Bias detection knob: {bias_detection_knob}. rmsre_prediction_cutoff: {rmsre_prediction_cutoff}. is_monotonic_scalar_query: {is_monotonic_scalar_query}. #queries {num_queries}")
cut_cdfs_fig

IndexError: index -1 is out of bounds for axis 0 with size 0

In [291]:
# chopped_df = df.query(f"is_monotonic_scalar_query == '{is_monotonic_scalar_query}'").copy()
chopped_df = df.copy()
chopped_df["rmsre_original"] = chopped_df["rmsre"]
chopped_df["rmsre"] = chopped_df.apply(apply_bias_detection, axis=1)
chopped_df.drop(columns=["rmsre_prediction"], inplace=True)

In [292]:
def fake_detection_knob(row):
    if row["baseline"] == "cookiemonster_bias":
        # return float(row["bias_detection_knob"])
        return str(row["bias_detection_knob"])
        
    if row["baseline"] == "cookiemonster":
        # return 0
        # return COOKIEMONSTER
        return str("N/A")
    if row["baseline"] == "cookiemonster_base":
        # return -1
        # return COOKIEMONSTER_BASE
        return str("N/A")
        
        
    if row["baseline"] == "ipa":
        # return -2
        # return IPA
        return str("N/A")
        

chopped_df["bias_detection_knob"] = chopped_df.apply(fake_detection_knob, axis=1)

In [293]:
rmsre_prediction_cutoffs = [0.05, 0.1, 0.2]
fake_cutoff_labels = [" ", "N/A", "  "]

dfs = []

for baseline, fake_cutoff_label in zip(DEFAULT_BASELINES_ORDER, fake_cutoff_labels):
    csv_name = csv_mapping[baseline] 
    base_df = query_df.query(f"baseline == '{csv_name}'").copy()
    base_df["rmsre_original"] = base_df["rmsre"]
    base_df.drop(columns=["rmsre_prediction"], inplace=True)
    # base_df["rmsre_prediction_cutoff"] = baseline
    base_df["rmsre_prediction_cutoff"] = fake_cutoff_label
    # base_df["rmsre_prediction_cutoff"] = "N/A"
    
    dfs.append(base_df)

for rmsre_prediction_cutoff in rmsre_prediction_cutoffs:
    chopped_df = query_df.query(f"baseline == 'cookiemonster_bias'").copy()
    chopped_df["rmsre_original"] = chopped_df["rmsre"]
    chopped_df["rmsre"] = chopped_df.apply(apply_bias_detection, axis=1)
    l = len(chopped_df.query("baseline == 'cookiemonster_bias'"))
    chopped_df.drop(columns=["rmsre_prediction"], inplace=True)
    chopped_df["rmsre_prediction_cutoff"] = str(rmsre_prediction_cutoff)
    
    dfs.append(chopped_df)
    
cutoffs_df = pd.concat(dfs, ignore_index=True)




In [294]:
cutoffs_df

,step,rmsre,user.baseline,user.bias_detection_knob,dataset.num_days_per_epoch,user.is_monotonic_scalar_query,is_monotonic_scalar_query,bias_detection_knob,num_days_per_epoch,baseline,rmsre_original,rmsre_prediction_cutoff
0,0,0.015612,ipa,0,7,True,True,0,7,ipa,0.015612,
1,1,0.015596,ipa,0,7,True,True,0,7,ipa,0.015596,
2,2,0.015604,ipa,0,7,True,True,0,7,ipa,0.015604,
3,3,0.015565,ipa,0,7,True,True,0,7,ipa,0.015565,
4,4,0.015510,ipa,0,7,True,True,0,7,ipa,0.015510,
...,...,...,...,...,...,...,...,...,...,...,...,...
2395,395,NaN,cookiemonster,0.5,7,True,True,0.5,7,cookiemonster_bias,0.216341,0.2
2396,396,NaN,cookiemonster,0.5,7,True,True,0.5,7,cookiemonster_bias,0.223221,0.2
2397,397,NaN,cookiemonster,0.5,7,True,True,0.5,7,cookiemonster_bias,0.223667,0.2
2398,398,NaN,cookiemonster,0.5,7,True,True,0.5,7,cookiemonster_bias,0.233721,0.2


In [295]:
variable = "rmsre_prediction_cutoff"
boxes_args = {
        "df": cutoffs_df,
        "metric": "rmsre",
        "x_axis": variable,
        "x_axis_title": "Error estimation cutoff",
        "y_axis_title": RMSRE_Y_LOG,
        "ordering": (variable, "str"),
        # "ordering": None,
        "showlegend": False,
        "baselines_order": BIAS_BASELINES_ORDER,
        "baselines_with_nqueries": BIAS_BASELINES_ORDER,
        "nqueries_style": "percentage",
        "hspace": 0,
        "vspace": 0.1,
        "log_y": True,
        "n_total_queries": num_queries,
        "y_range": y_range,
        
    }

cutoff_boxes_fig = go.Figure()
# fig.update_yaxes(type="log")
for trace in boxes(**boxes_args):
    cutoff_boxes_fig.add_trace(trace)
cutoff_boxes_fig.update_layout(title=f"Quartiles for RMSRE depending on rmsre_prediction_cutoff. bias detection knob: {bias_detection_knob}. is_monotonic_scalar_query: {is_monotonic_scalar_query}.")
cutoff_boxes_fig

In [296]:

data = []
global_metric_names = ["avg_budget"]
param_names = ["user.baseline", "user.bias_detection_knob", "dataset.num_days_per_epoch"]


for run_id in run_ids:
    datapoint = {}

    try:
        run_params = client.get_run(run_id).data.params
        for param_name in param_names:
            datapoint[param_name] = run_params[param_name]
            
        for metric_name in global_metric_names:
            global_metric = client.get_metric_history(run_id, metric_name)[0]        
            datapoint[metric_name] = global_metric.value        
    except Exception as e:
        print(e)
        continue
        
    data.append(datapoint)
    
budget_df = pd.DataFrame(data)
budget_df["baseline"] = budget_df.apply(get_baseline_name, axis=1)
budget_df["bias_detection_knob"] = budget_df["user.bias_detection_knob"]
budget_df["num_days_per_epoch"] = budget_df["dataset.num_days_per_epoch"]


In [297]:
budget_df

,user.baseline,user.bias_detection_knob,dataset.num_days_per_epoch,avg_budget,baseline,bias_detection_knob,num_days_per_epoch
0,cookiemonster,0,7,0.358504,cookiemonster,0,7
1,ipa,0,7,0.967086,ipa,0,7
2,cookiemonster_base,0,7,0.564632,cookiemonster_base,0,7
3,cookiemonster,0.5,7,0.428124,cookiemonster_bias,0.5,7
4,cookiemonster_base,0.25,7,0.564443,cookiemonster_base_bias,0.25,7
5,cookiemonster,0.25,7,0.394500,cookiemonster_bias,0.25,7
6,cookiemonster_base,1,7,0.579929,cookiemonster_base_bias,1,7
7,cookiemonster,1,7,0.495085,cookiemonster_bias,1,7
8,cookiemonster_base,0.5,7,0.561484,cookiemonster_base_bias,0.5,7


In [298]:

bars_args = {
            "df": budget_df.query(f"bias_detection_knob == '0' or bias_detection_knob == '{bias_detection_knob}'"),
            "metric": "avg_budget",
            "x_axis": "num_days_per_epoch",
            "x_axis_title": "",
            "y_axis_title": BUDGET_CONSUMPTION_Y_AVG,
            # "ordering": ("rmsre_prediction_cutoff", "str"),
            "ordering": None,
            "log_y": False,
            "showlegend": False,
            "baselines_order": BIAS_BASELINES_ORDER,
            "y_range": [0,1],
        }
budget_fig = go.Figure()
for trace in bars(**bars_args):
    budget_fig.add_trace(trace)
budget_fig.update_layout(title=f"Quartiles for RMSRE depending on rmsre_prediction_cutoff. bias detection knob: {bias_detection_knob}. is_monotonic_scalar_query: {is_monotonic_scalar_query}.")
budget_fig

In [299]:
def make_bias_plots(
    args,
    cols,
    legend,
    axis_title_font_size,
    axis_tick_font_size,
    output_path,
    height=None,
    width=None,
    titles=None,
):
    # bars_args, cdf_args, cut_cdf_args, boxes_args = args
    # column_widths = [0.25* 0.3, 0.25, 0.25, 0.25]  # First subplot is smaller

    bars_args, cdf_args, boxes_args = args
    column_widths = [0.2, 0.4, 0.4]  # First subplot is smaller

    fig = make_subplots(
        rows=1, cols=cols, specs=[[{"type": "xy"}] * cols], horizontal_spacing=0.08, subplot_titles=titles,
        column_widths=column_widths,row_heights=[1]
    )
    
    for trace in bars(**bars_args):
            fig.add_trace(trace, row=1, col=1)
            
    fig.update_xaxes(
        showticklabels=False,  # Remove tick labels
        row=1,
        col=1
    )
    

    for trace in cdf(**cdf_args):
        fig.add_trace(trace, row=1, col=2)
        
    fig.add_trace(trace_unsorted, row=1, col=2)
    # fig.add_trace(trace_cutoff, row=1, col=2)        
        
    # for trace in cdf(**cut_cdf_args):
    #     fig.add_trace(trace, row=1, col=3)
        
    for trace in boxes(**boxes_args):
        fig.add_trace(trace, row=1, col=3)
    fig.update_xaxes(
        categoryorder='array',
        categoryarray=fake_cutoff_labels + rmsre_prediction_cutoffs,
        row=1,
        col=3
    )
    


    for i in range(cols):
        fig.update_xaxes(
            title=args[i].get("x_axis_title"),
            tickfont=dict(size=axis_tick_font_size.get("x")),
            title_font=dict(size=axis_title_font_size.get("x")),
            showgrid=True,
            range=args[i].get("x_range"),
            row=1,
            col=i + 1,
        )
        fig.update_yaxes(
            title=args[i].get("y_axis_title"),
            title_standoff=0,
            tickfont=dict(size=axis_tick_font_size.get("y")),
            title_font=dict(size=axis_title_font_size.get("y")),
            type="log" if args[i].get("log_y") else "linear",
            range=args[i].get("y_range"),
            showgrid=True,
            row=1,
            col=i + 1,
            domain=[0, 1], # Use more space
        )

    fig.update_layout(
        legend={
            "title": None,
            "font": {"size": 20},
            # "traceorder": "reversed",
            **legend,
        },
        template=TEMPLATE,
        showlegend=True,
        height=height,
        width=width,
        # barmode="group",
        # boxmode="group",
        margin=dict(t=0, b=0),
    )
    fig.write_image(f"{output_path}", engine="kaleido")
    fig.show()



In [300]:
figs_args = {
    "axis_title_font_size": {"x": 18, "y": 18},
    "axis_tick_font_size": {"x": 14, "y": 14},
    "legend": {
        "yanchor": "bottom",
        "y": 1.1,
        # "xanchor": "left",
        # "x": 0.2,
        "orientation": "h",
    },
    "output_path": "bias_plots.pdf",
    "height": 300,
    "width": 1500,
}
# make_plots(
#            [full_cdfs, cut_cdfs_fig, cutoff_boxes_fig, budget_fig], cols=4, **figs_args)

make_bias_plots(
           [bars_args, cdf_args, boxes_args], cols=3, **figs_args)

NameError: name 'trace_unsorted' is not defined